In [1]:
import os.path
from datetime import datetime
import requests
import json

# lekerdezes
# APPID regisztracioval igenyelheto, ingyenes fiokkal 10 percnel gyakrabban nem szabad
# lekerdezni, kulonben 1 orara letiltjak a fiokot. Ezt a szkript ellenorzi, es ha az
# elmult 10 percben tortent lekerdezes, akkor annak az eredmenyet jeleniti meg.
timeFormat='%Y-%m-%d %H:%M:%S.%f'
if not os.path.exists('lastRequest.txt'):
    lastRqFile=open('lastRequest.txt','w')
    lastRequest='2000-01-01 00:00:00.000000'
else:
    lastRqFile=open('lastRequest.txt','r+')
    lastRequest=lastRqFile.read()
curRequest=datetime.now().strftime(timeFormat)
diff=datetime.strptime(curRequest,timeFormat)-datetime.strptime(lastRequest,timeFormat)
if diff.total_seconds()>660:
    lastRqFile.seek(0)
    lastRqFile.truncate()
    lastRqFile.write(curRequest)
    lastRequest=curRequest
    with open('appid.txt') as appidFile:
        appid=appidFile.read()
        appidFile.close()
    req=requests.get('https://api.openweathermap.org/data/2.5/weather?', \
                     params={'id':'7284824','appid':appid})
    if req.status_code!=200:
        raise Exception('HTTP error, code %d.' % req.status_code)
    with open('weather.json','w') as jsonFile:
        jsonFile.write(str(req.text))
        jsonFile.close()
print('Time of request: %s' % datetime.strptime(lastRequest,timeFormat))
lastRqFile.close()

# JSON file feldolgozasa
with open('weather.json') as jsonFile:
    data=json.loads(jsonFile.read())
    jsonFile.close()
print('City:            %s' % data['name'])
print('GPS location:    %f, %f' % (data['coord']['lon'],data['coord']['lat']))
print('Weather:         %s (%s)' % (data['weather'][0]['main'],data['weather'][0]['description']))
print('Temperature:     %.2f %cC' % (data['main']['temp']-273.15,chr(176)))
print('Humidity:        %.f%%' % data['main']['humidity'])
print('Pressure:        %.f hPa' % data['main']['pressure'])
print('Wind:            %.1f m/s' % (data['wind']['speed']))
print('Visibility:      %.f m' % data['visibility'])
print('Sunrise:         %s' % str(datetime.fromtimestamp(data['sys']['sunrise'])))
print('Sunset:          %s' % str(datetime.fromtimestamp(data['sys']['sunset'])))

Time of request: 2018-08-15 02:04:49.170199
City:            Budapest XI. keruelet
GPS location:    19.040000, 47.480000
Weather:         Clear (clear sky)
Temperature:     19.00 °C
Humidity:        100%
Pressure:        1013 hPa
Wind:            1.0 m/s
Visibility:      10000 m
Sunrise:         2018-08-15 05:39:36
Sunset:          2018-08-15 19:56:10
